In [66]:
#import the necessary functions
import numpy as np 
from math import sin,cos
import modern_robotics as mr

In [41]:
def inputValidation(a,b,c,d,e):
    #SIZE CHECKS
    #make sure currentConfig is (12,)
    if isinstance(a,np.ndarray) != True:
        raise ValueError("Input needs to be an np.array")
    elif(a.shape != (12,)):
        raise ValueError("Input a is not the right size.") 
    
    #make sure u is (9,)
    if isinstance(b,np.ndarray) != True:
        raise ValueError("Input needs to be an np.array")
    elif(b.shape != (9,)):
        raise ValueError("Input b is not the right size.") 
    
    #make sure that dt is an int or float
    if isinstance(c, int | float) != True:
        raise ValueError("c is not an int or float")
    
    #make sure armSpeed is an int or float
    if isinstance(d, int | float) != True:
        raise ValueError("d is not an int or float")
    
    #makesure wheelSpeed in an int or float
    if isinstance(e, int | float) != True:
        raise ValueError("e is not an int or float")
    
    return 

#TEST CASES
a = np.ones((12))
b = np.ones((9))
c = "hello"
d = 1
e = 2
inputValidation(a,b,c,d,e) #should pass 


ValueError: c is not an int or float

In [89]:
def NextState(currentConfig, u, dt, maxjointSpeed, maxwheelSpeed):
    '''
    NextState will simulate the kinematics for the youBot. 

    Parameters
    ----------
    currentConfig   : np.array
        a (12,) vector representing the current configuration of the robot. 
        i.e [chassis, arm, wheel angles] 

    u               : np.array
        a (9,) vector representing the wheel speeds and the arm speeds.
        i.e [u_1 , u_2, u_3, u_4, tdot_1, tdot_2, tdot_3, tdot_4]

    dt              : float
        a number that represents the timestep being taken. 

    maxjointSpeed        : float
        a number that represents the maximum angular speed of the robot arm joints. 

    maxwheelSpeed      : float
        a number that represents the maximum wheel speed of the robot.

    Outputs
    -------
    newConfig: np.array
        a 12, vector representing the configuration of the robot dt seconds later.
    '''

    #INPUT VARIABLE CHECKS

    #SPLIT OUR INPUTS INTO USEABLE VECTORS
    #currentConfig breakdown
    chassisConfig = currentConfig[0:3]
    armConfig = currentConfig[3:8]
    wheelAngle = currentConfig[8:]

    #u breakdown
    wheelSpeed = u[0:4]
    jointSpeed = u[4:]

    #make sure that we are not over the maximum limits. 
    wheelSpeed[wheelSpeed > maxwheelSpeed] = maxwheelSpeed
    wheelSpeed[wheelSpeed < -1 * maxwheelSpeed] = -1 * maxwheelSpeed
    jointSpeed[jointSpeed < -1 * maxjointSpeed] = -1 * maxjointSpeed
    jointSpeed[jointSpeed > maxjointSpeed] = maxjointSpeed

    #UPDATE THE NEW JOINT ANGLES + WHEEL ANGLES
    newarmConfig = armConfig + jointSpeed * dt 
    newwheelAngle = wheelAngle + wheelSpeed * dt 

    #HARDCODED KINEMATICS
    r = 0.0475
    l = 0.235
    w = 0.15
    F = (r/4) * np.array([[-1/(l+w), 1/(l+w), 1/(l+w), -1/(l+w)],
                          [       1,       1,       1,        1],
                          [      -1,       1,      -1,        1]])
    
    #UPDATE THE NEW CHASSIS CONFIGURATION
    #find the chassis planar twist
    Vb = F @ (wheelSpeed * dt)
    w_bz = Vb[0]
    v_bx = Vb[1]
    v_by = Vb[2]

    #find the new dq
    if w_bz == 0:
        dq_b = np.array([0, v_bx, v_by])
    else: 
        dq_b = np.array([w_bz,
                         (v_bx * sin(w_bz) + v_by * (cos(w_bz) - 1))/w_bz,
                         (v_by * sin(w_bz) + v_bx * (1 - cos(w_bz)))/w_bz
                         ])
        
    #transform the dq_b to dq_s 
    phi_k = currentConfig[0]
    rotMat = np.array([[1,          0,           0],
                       [0, cos(phi_k), -sin(phi_k)],
                       [0, sin(phi_k),  cos(phi_k)]])
    
    dq_s = rotMat @ dq_b

    newchassisConfig = chassisConfig + dq_s

    #COMPILE ALL THE CONFIGURATIONS
    newConfig = np.hstack((newchassisConfig, newarmConfig, newwheelAngle))

    return newConfig

currentConfig= np.zeros((12))
u= np.array([10,10,10,10,0,0,0,0,0])
dt= 0.01
maxarmSpeed= 5
maxwheelSpeed = 5

curr = np.hstack((NextState(currentConfig, u, dt, maxarmSpeed, maxwheelSpeed), 0))[None,:]
print(curr[-1,0:12].shape)
for i in range(100): 
    gripper = np.hstack((NextState(curr[-1,0:12], u, dt, maxarmSpeed, maxwheelSpeed), 0))
    curr = np.vstack((curr, gripper ))

np.savetxt('theta' + '.csv', curr, delimiter=',')

(12,)
